In [1]:
import numpy as np
import tncontract as tn
import qutip as qt
import time

import sys
from itertools import product
sys.path.append("../src/")
from simulation_utils import *

In [ ]:
n_sites = 20 # number of sites
psi = tn.onedim.init_mps_allzero(n_sites, 2) # initial state all zeros
cct_depth = 100

# limit the bond dimension
# chi = int(2 * np.log2(n_sites))
chi=None

t = time.time()

for _ in range(int(cct_depth/3)):
    # two qubit layer
    cct_layer_1, cct_layer_2 = random_two_qubit_gate_ladder(n_sites) 
    psi = tn.onedim.contract_mps_mpo(psi,cct_layer_1)
    psi.left_canonise(chi=chi)
    psi = tn.onedim.contract_mps_mpo(psi,cct_layer_2)
    psi.left_canonise(chi=chi)
    
    # single qubit layer
    cct_layer = random_single_qubit_gate_layer(n_sites)
    psi = tn.onedim.contract_mps_mpo(psi,cct_layer)
    psi.left_canonise(chi=chi)

psi.left_canonise(normalise=True)
print("Took {:.2f}s to run {} circuits on {} qubits".format(time.time()-t, cct_depth, n_sites))

In [ ]:
for s in psi:
    print(s.shape)

In [18]:
psi_vec = tn.onedim.contract_virtual_indices(psi)

In [19]:
psi_vec.fuse_indices('physout','physout')

In [20]:
psi_vec.shape

(512,)

In [21]:
qt.Qobj(psi_vec.data.reshape(-1,1))

Quantum object: dims = [[512], [1]], shape = (512, 1), type = ket
Qobj data =
[[-2.90078115e-02-3.43452109e-02j]
 [-2.89272709e-02+4.06069157e-02j]
 [ 1.78715908e-02+6.63665775e-02j]
 [-2.72705922e-02-3.75427704e-02j]
 [ 5.11022241e-03+7.51369457e-03j]
 [ 2.30360616e-02+5.92988331e-02j]
 [ 3.39413971e-02-7.08968092e-02j]
 [ 9.84344656e-03-1.28670128e-02j]
 [-3.49260000e-03+4.16729687e-02j]
 [ 4.65110309e-02+8.35551554e-03j]
 [ 3.22995665e-02-6.33085758e-02j]
 [ 3.19229776e-02+3.79871468e-02j]
 [ 2.79056401e-02+6.89121852e-02j]
 [ 2.59092170e-02-5.83718207e-03j]
 [ 2.62038444e-02+3.18462753e-02j]
 [-2.90604427e-02-1.38435310e-02j]
 [ 7.58960323e-02+6.09671569e-02j]
 [ 3.48660158e-02+1.37125198e-02j]
 [ 6.24859670e-02-2.57283648e-02j]
 [ 2.43820997e-02+1.91701990e-02j]
 [ 9.84987101e-02-1.70988907e-02j]
 [ 2.14312754e-02-5.90607711e-02j]
 [-4.71974429e-02+2.26025548e-02j]
 [-4.55877461e-02-5.46925791e-02j]
 [ 2.44412301e-02-8.73374185e-03j]
 [ 3.40835246e-02-6.27827889e-02j]
 [-3.1407857